# 预处理

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
# 下载数据集
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# MNIST数据集，输入为28*28的图片，除以255为归一化
input_x = tf.placeholder(tf.float32,[None,28*28])/255
# 输出为0-9十个数字
output_y = tf.placeholder(tf.int32,[None,10])

In [4]:
input_x_images = tf.reshape(input_x,[-1,28,28,1])

# 创建CNN

In [5]:
conv1 = tf.layers.conv2d(inputs=input_x_images,filters=32,kernel_size=[5,5],strides=1,padding='same',activation=tf.nn.relu)

In [6]:
pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)

In [7]:
conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],strides=1,padding='same',activation=tf.nn.relu)

In [8]:
pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)

In [9]:
flat = tf.reshape(pool2,[-1,7*7*64])

In [10]:
dense = tf.layers.dense(inputs=flat,units=1024,activation=tf.nn.relu)

In [11]:
dropout = tf.layers.dropout(inputs=dense,rate=0.8)

In [12]:
logits = tf.layers.dense(inputs=dropout,units=10)

# 最小化损失函数

In [13]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=output_y,logits=logits)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [14]:
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [15]:
accuracy = tf.metrics.accuracy(labels=tf.argmax(output_y,axis=1),predictions=tf.argmax(logits,axis=1),)[1]

# 训练CNN模型

In [16]:
i = 0
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

while i < 1001:
    batch = mnist.train.next_batch(50) 
    train_loss,train_op_ = sess.run([loss,train_op],{input_x:batch[0],output_y:batch[1]})
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy,{input_x:batch[0],output_y:batch[1]})
        print("第 {} 步的 训练损失={:.4f}, 测试精度={:.5f}".format(i, train_loss, train_accuracy))
    i += 1

第 0 步的 训练损失=2.3268, 测试精度=0.34000
第 100 步的 训练损失=0.1869, 测试精度=0.65000
第 200 步的 训练损失=0.0477, 测试精度=0.76667
第 300 步的 训练损失=0.1112, 测试精度=0.81500
第 400 步的 训练损失=0.2476, 测试精度=0.84400
第 500 步的 训练损失=0.0282, 测试精度=0.87000
第 600 步的 训练损失=0.1499, 测试精度=0.88571
第 700 步的 训练损失=0.2332, 测试精度=0.89500
第 800 步的 训练损失=0.0092, 测试精度=0.90667
第 900 步的 训练损失=0.0484, 测试精度=0.91600
第 1000 步的 训练损失=0.0511, 测试精度=0.92182


In [17]:
test_x = mnist.test.images
test_y = mnist.test.labels

In [18]:
test_output = sess.run(logits,{input_x:test_x[:50]})

In [19]:
inferred_y = np.argmax(test_output,1)

In [20]:
test_accuracy = sess.run(accuracy,{input_x:test_x[:1000],output_y:test_y[:1000]})

In [21]:
print('推测的数字:', inferred_y)  # 推测的数字
print('真实的数字:', np.argmax(test_y[:50], 1))  # 真实的数字
print('测试精度:{:.5f}'.format(test_accuracy))  # 识别准确度

推测的数字: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4]
真实的数字: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4]
测试精度:0.96645
